## Import packages

In [25]:
import selenium
from selenium import webdriver

import shutil, time, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Matth\\git\\DataAnalysisWorkbooks\\warcraftLogs\\src.py'>

## Configurations

In [26]:
# Set path variables
path_to_adblocker = r"C:\Users\Matth\Desktop\3.12_0"
path_to_download_dir = r"C:\Users\Matth\Downloads"

In [23]:
# Set character info
character_name = "Mercychan"
character_server = "Benediction"
character_region = "US"

## Navigate to character page

In [24]:
char_url, browser = src.load_environment(path_to_adblocker, character_name, character_server, character_region)

In [15]:
bosses = src.get_t6_bosses(browser)
bosses

["High Warlord Naj'entus",
 'Supremus',
 'Shade of Akama',
 'Teron Gorefiend',
 'Gurtogg Bloodboil',
 'Reliquary of Souls',
 'Mother Shahraz',
 'The Illidari Council',
 'Illidan Stormrage',
 'Rage Winterchill',
 'Anetheron',
 "Kaz'rogal",
 'Azgalor',
 'Archimonde']

In [5]:
# Alternatively specify specific bosses
bosses = ["Rage Winterchill"]

In [6]:
# Dictionary for browser to navigate to boss page URLs
boss_link_dict = {"High Warlord Naj'entus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=618", "Anetheron" : "#boss=619", "Kaz'rogal" : "#boss=620", 
                  "Azgalor" : "#boss=621", "Archimonde" : "#boss=622"}

In [10]:
char_info = src.get_character_info(char_url)
char_info

'Mercychan (Benediction US)'

## Main code loop

In [ ]:
def main():
    
    for boss in bosses:
        
        browser.get(f"https://classic.warcraftlogs.com/character/{character_region.lower()}/{character_server.lower()}/{character_name.lower()}{boss_link_dict[boss]}")
        time.sleep(1)

        # Pick out the top parse of this boss
        search = browser.find_elements_by_class_name("character-table-link")
        boss_page_url = browser.current_url
        
        print(f"Boss: {boss}")
        
        #for i in range(len(search)):
        for i in range(3):
            
            player_df = pd.DataFrame(pd.np.empty((0, 16)))
            player_df.columns = ["Name", "Server", "Date", "Kill time", "Rank", "nHealers", "Spriest?", "Innervate?", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotations"]
                        
            try:
                rank, date, boss_rank, boss_HPS, boss_killtime, HPS_link = src.get_boss_data(browser, i)
                
            except IndexError: 
                continue
                
            search = browser.find_elements_by_class_name("character-table-link")
                
            if src.check_if_parse_already_recorded(i, browser, search, boss, character_name, character_server, character_region): 
                print("Parse already recorded. Skipping")
                browser.get(boss_page_url)
                continue
            
            HPS_link.click()
            time.sleep(2)
            
            temp_url = browser.current_url
                        
            boss_tanks = src.get_tanks(browser)
            nHealers = src.get_nHealers(browser)
            
            print(f"Tanks: {boss_tanks}")

            browser.get(temp_url)
            time.sleep(0.5)

            #src.scroll_click(browser.find_element_by_link_text(character_name))
            c = browser.find_element_by_link_text(character_name)
            c.click()
            time.sleep(0.5)
            
            LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, boss_HPS)
            time.sleep(2)
                
            spriest = src.check_spriest(browser)
            innervate = src.check_innervate(browser)

            # Download the cast-sequence and damage-taken CSVs.
            src.download_csv(browser, temp_url, "filter-casts-tab", path_to_download_dir, "character_data/cast_sequence.csv")
            src.download_csv(browser, temp_url, "filter-damage-taken-tab", path_to_download_dir, "character_data/dmg_taken.csv")

            #nHealers = src.get_nHealers(browser)
            browser.get(temp_url)
            time.sleep(1)

            # For most single tank bosses, the main tank is the current boss' melee target
            if boss in ["Rage Winterchill", "Kaz'rogal", "Archimonde", "High Warlord Naj'entus", "Supremus", 
                        "Teron Gorefiend", "Gurtogg Bloodboil", "Mother Shahraz"]:

                # Clean the csvs
                df = src.clean_dmg_taken_csv(boss)
                df2 = src.clean_cast_sequence_csv()

                # Fix regrowth timer
                df2 = src.fix_cast_time(df2)

                # Add boss target
                df = src.add_boss_target(boss, df, df2)
                df = src.fix_boss_target(df)

                # Get the rotations
                rotations = src.calculate_rotations(df, boss, boss_tanks)


            else:
                # Clean the csv
                df = src.clean_cast_sequence_csv()
                df = src.fix_cast_time(df)

                # Get the rotations
                rotations = src.calculate_rotations(df, boss, boss_tanks)
                
                
            print(f'Rotations: {rotations}')    

            # Add data to pandas dataframe
            to_append = [character_name, character_server + " " + character_region, date, boss_killtime, boss_rank, str(nHealers), spriest, innervate, LB_uptime, boss_HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotations]
            series = pd.Series(to_append, index = player_df.columns)
            player_df = player_df.append(series, ignore_index=True)
            
            # Export scraped data to csv and add it to the excel spreadsheet
            player_df.to_csv(f"character_data/{boss.replace(' ', '')}_{character_name}.csv", index=None)
            
            src.combine_character_data(character_name, character_server, character_region, date, boss_rank, boss)
            src.add_row_to_xlsx(boss, character_name)
            src.sort_excel(boss)
            
            os.remove(f"character_data/{boss.replace(' ', '')}_{character_name}.csv")
            
            #Return to boss page
            browser.get(boss_page_url)
            time.sleep(1)  # Cannot be < 1s
            print('------')

        print(f"{boss} scraping complete.")
        print("---------------")
        
     # Return to character page
    browser.get(char_url)
    

if __name__ == "__main__":
    main()

Boss: Rage Winterchill
Parse already recorded. Skipping
Parse already recorded. Skipping
Tanks: ['Armadomin', 'Hilfedanke', 'Formulatank']
